<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install syft

In [ ]:
import syft as sy
import torch
from torch import nn, optim

# Hook PyTorch
hook = sy.TorchHook(torch)

# Define virtual workers
client1 = sy.VirtualWorker(hook, id="client1")
client2 = sy.VirtualWorker(hook, id="client2")

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        return self.fc(x)

# Define the federated training function
def train_on_client(client, model, optimizer, loss_fn, data, target):
    # Send the model and data to the client
    model.send(client)
    data, target = data.send(client), target.send(client)

    # Perform a forward pass and compute loss
    optimizer.zero_grad()
    predictions = model(data)
    loss = loss_fn(predictions, target)

    # Backpropagation and optimizer step
    loss.backward()
    optimizer.step()

    # Get the updated model back to the central server
    model.get()
    return loss.get().item()

# Instantiate the model, optimizer, and loss function
model = SimpleNN()
optimizer = optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# Example datasets for clients
data1 = torch.tensor([[1.0, 2.0], [2.0, 3.0]])
target1 = torch.tensor([[3.0], [5.0]])
data2 = torch.tensor([[3.0, 4.0], [4.0, 5.0]])
target2 = torch.tensor([[7.0], [9.0]])

# Federated training loop
for epoch in range(5):  # Train for 5 epochs
    loss1 = train_on_client(client1, model, optimizer, loss_fn, data1, target1)
    loss2 = train_on_client(client2, model, optimizer, loss_fn, data2, target2)
    print(f"Epoch {epoch + 1}, Client 1 Loss: {loss1:.4f}, Client 2 Loss: {loss2:.4f}")